In [25]:
import numpy as np
import torch
import pandas as pd
import torchvision
from torchvision import datasets,transforms
from torch import nn
from torch.nn import functional as F
from torchsummary import summary

transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                               transforms.Normalize(mean=[0.5,],std=[0.5,])])

data_train = datasets.FashionMNIST(root="./data",
                                   transform=transform,
                                   train=True,
                                   download=True)
data_test = datasets.FashionMNIST(
    root="./data",
    transform=transform,
    train=False
)

data_loader_train = torch.utils.data.DataLoader(dataset=data_train,
                                                batch_size=256,
                                                shuffle=True)
data_loader_test = torch.utils.data.DataLoader(dataset=data_test,
                                               batch_size=256,
                                               shuffle=True)
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,96,kernel_size=11,stride=4,padding=1)
        self.conv2 = nn.Conv2d(96,256,kernel_size=5,padding=2)
        self.conv3 = nn.Conv2d(256,384,kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(384,384,kernel_size=3,padding=1)
        self.conv5 = nn.Conv2d(384,256,kernel_size=3,padding=1)
        self.fc1 = nn.Linear(6400,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),3,2)
        x = F.max_pool2d(F.relu(self.conv2(x)),3,2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x,3,2)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x,0.5)
        x = F.relu(self.fc3(x))
net = Net()
summary(net,(1,224,224))
lr = 0.5
num_epochs = 10
criterion = F.nll_loss
optimizer = torch.optim.Adam(net.parameters())
loss_fn = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    net.train()
    for idx,(train_x,train_label) in enumerate(data_loader_train):
        optimizer.zero_grad()
        pred_y = net(train_x)
        loss = criterion(pred_y,train_label)
        if idx % 10000 == 0:
            print("loss:{:.4f}".format(loss))
        loss.backward()
        optimizer.step()
    all_correct_num = 0
    all_sample_num = 0
    net.eval()
    for i, (test_x,test_label) in enumerate(data_loader_test):
        pred_y = net(test_x.float()).detach()
        pred_y = np.argmax(pred_y,axis=-1)
        current_correct_num = pred_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(),axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print("epoch {:d}: acc = {:.4f}".format(epoch,acc))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
            Conv2d-2          [-1, 256, 26, 26]         614,656
            Conv2d-3          [-1, 384, 12, 12]         885,120
            Conv2d-4          [-1, 384, 12, 12]       1,327,488
            Conv2d-5          [-1, 256, 12, 12]         884,992
            Linear-6                 [-1, 4096]      26,218,496
            Linear-7                 [-1, 4096]      16,781,312
            Linear-8                   [-1, 10]          40,970
Total params: 46,764,746
Trainable params: 46,764,746
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 4.64
Params size (MB): 178.39
Estimated Total Size (MB): 183.23
----------------------------------------------------------------
torch.Size([256, 1, 224, 224])


TypeError: nll_loss_nd(): argument 'input' (position 1) must be Tensor, not NoneType